<a href="https://colab.research.google.com/github/jiwonlydia/return_prediction/blob/main/0613_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공식 깃허브: https://github.com/LouisChen1992/Deep_Learning_Asset_Pricing

# Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/research/return_prediction/dlap'
os.chdir(path)

In [3]:
pip install tensorflow==1.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 43 kB/s 
     |████████████████████████████████| 488 kB 10.5 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 3.1 MB 27.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
ERROR: pip's dependency resolver does not currently ta

# Config

In [4]:
import os
import json
import numpy as np

import tensorflow as tf
print(tf.__version__)

from src.data import data_layer
from src.utils import deco_print

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
#### Delete all flags before declare #####

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

- tensorflow의 flag란?
- https://daeson.tistory.com/256

In [6]:
# del_all_flags(tf.flags.FLAGS)

tf.flags.DEFINE_string("f", "", "kernel") # 주피터 노트북에서 돌릴땐 이 코드 추가해야 돌아감

tf.flags.DEFINE_string('config', path+'/config/config.json', 'Path to the file with configurations')
tf.flags.DEFINE_string('logdir', '', 'Path to save logs and checkpoints')

tf.flags.DEFINE_integer('saveBestFreq', -1, 'Frequency to save best model')
tf.flags.DEFINE_boolean('printOnConsole', True, 'Print on console or not')
tf.flags.DEFINE_boolean('saveLog', True, 'Save log or not')
tf.flags.DEFINE_integer('printFreq', 128, 'Frequency to print on console')
tf.flags.DEFINE_integer('ignoreEpoch', 64, 'Ignore first several epochs')

FLAGS = tf.flags.FLAGS
print(FLAGS.config)
print(FLAGS.logdir)
print(FLAGS.saveBestFreq)
print(FLAGS.printOnConsole)
print(FLAGS.saveLog)
print(FLAGS.printFreq)
print(FLAGS.ignoreEpoch)

/content/drive/MyDrive/research/return_prediction/dlap/config/config.json

-1
True
True
128
64


In [7]:
with open(FLAGS.config, 'r') as file:
    config = json.load(file)
    if not 'macro_idx' in config:
        config['macro_idx'] = None

deco_print('Read the following in config: ')
print(json.dumps(config, indent=4))

>==================> Read the following in config: 
{
    "learning_rate": 0.001,
    "num_layers_moment": 0,
    "num_units_rnn": [
        4
    ],
    "macro_feature_file_test": "datasets/macro/macro_test.npz",
    "optimizer": "Adam",
    "macro_feature_file": "datasets/macro/macro_train.npz",
    "use_rnn": true,
    "weighted_loss": true,
    "hidden_dim": [
        64,
        64
    ],
    "cell_type_rnn_moment": "lstm",
    "cell_type_rnn": "lstm",
    "macro_feature_file_valid": "datasets/macro/macro_valid.npz",
    "num_epochs_moment": 64,
    "tSize_test": 300,
    "tSize": 240,
    "individual_feature_dim": 46,
    "tSize_valid": 60,
    "num_condition_moment": 8,
    "loss_factor": 1.0,
    "num_layers_rnn_moment": 1,
    "individual_feature_file_valid": "datasets/char/Char_valid.npz",
    "num_epochs": 1024,
    "individual_feature_file": "datasets/char/Char_train.npz",
    "num_units_rnn_moment": [
        32
    ],
    "num_epochs_unc": 256,
    "dropout": 0.95,
    "m

In [8]:
config

{'cell_type_rnn': 'lstm',
 'cell_type_rnn_moment': 'lstm',
 'dropout': 0.95,
 'hidden_dim': [64, 64],
 'hidden_dim_moment': [],
 'individual_feature_dim': 46,
 'individual_feature_file': 'datasets/char/Char_train.npz',
 'individual_feature_file_test': 'datasets/char/Char_test.npz',
 'individual_feature_file_valid': 'datasets/char/Char_valid.npz',
 'learning_rate': 0.001,
 'loss_factor': 1.0,
 'macro_feature_dim': 178,
 'macro_feature_file': 'datasets/macro/macro_train.npz',
 'macro_feature_file_test': 'datasets/macro/macro_test.npz',
 'macro_feature_file_valid': 'datasets/macro/macro_valid.npz',
 'macro_idx': None,
 'num_condition_moment': 8,
 'num_epochs': 1024,
 'num_epochs_moment': 64,
 'num_epochs_unc': 256,
 'num_layers': 2,
 'num_layers_moment': 0,
 'num_layers_rnn': 1,
 'num_layers_rnn_moment': 1,
 'num_units_rnn': [4],
 'num_units_rnn_moment': [32],
 'optimizer': 'Adam',
 'sub_epoch': 4,
 'tSize': 240,
 'tSize_test': 300,
 'tSize_valid': 60,
 'use_rnn': True,
 'weighted_loss': 

# Load Data

In [9]:
dl = data_layer.DataInRamInputLayer(
		config['individual_feature_file'],
		pathMacroFeature=config['macro_feature_file'],
		macroIdx=config['macro_idx']) # 0 ~ 177
	
# normalize 하기 위해 macro 변수의 mean, std 계산
meanMacroFeature, stdMacroFeature = dl.getMacroFeatureMeanStd()

dl_valid = data_layer.DataInRamInputLayer(
    config['individual_feature_file_valid'],
    pathMacroFeature=config['macro_feature_file_valid'],
    macroIdx=config['macro_idx'], 
    meanMacroFeature=meanMacroFeature, # train에서 계산한 값으로 mean 설정
    stdMacroFeature=stdMacroFeature) # train에서 계산한 값으로 std 설정

dl_test = data_layer.DataInRamInputLayer(
    config['individual_feature_file_test'],
    pathMacroFeature=config['macro_feature_file_test'],
    macroIdx=config['macro_idx'], 
    meanMacroFeature=meanMacroFeature, # train에서 계산한 값으로 mean 설정
    stdMacroFeature=stdMacroFeature) # train에서 계산한 값으로 std 설정

if config['weighted_loss']: # True
    # 회사 3686개에 대해 return값이 존재하는 날짜의 개수를 합함 (axis=0) => 회사별 weight
    loss_weight = dl.getDateCountList() # shape (3686,)
    loss_weight_valid = dl_valid.getDateCountList()
    loss_weight_test = dl_test.getDateCountList()
else:
    loss_weight = None
    loss_weight_valid = None
    loss_weight_test = None
deco_print('Data layer created')


>==================> Data layer created


In [11]:
model_params = config

# LSTM
---
- https://wdprogrammer.tistory.com/34

In [12]:
from tensorflow.python.ops.rnn_cell import BasicRNNCell
from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper
from tensorflow.python.ops.rnn_cell import MultiRNNCell

def create_rnn_cell(cell_type, num_units, num_layers=1, dp_input_keep_prob=1.0, dp_output_keep_prob=1.0):
	def single_cell(num_units):
		if cell_type == 'rnn':
			cell_class = BasicRNNCell
		elif cell_type == 'gru':
			cell_class = GRUCell
		elif cell_type == 'lstm':
			cell_class = LSTMCell
		else:
			raise ValueError('Cell Type Not Supported! ')

		if dp_input_keep_prob != 1.0 or dp_output_keep_prob != 1.0:
			return DropoutWrapper(cell_class(num_units=num_units),
				input_keep_prob=dp_input_keep_prob,
				output_keep_prob=dp_output_keep_prob)
		else:
			return cell_class(num_units=num_units)

	assert(len(num_units) == num_layers)
	if num_layers > 1:
		return MultiRNNCell([single_cell(num_units[i]) for i in range(num_layers)])
	else:
		return single_cell(num_units[0])

In [65]:
def initial_state_size(cell_type, num_units):
	state_size = sum(num_units)
		return state_size * 2

In [139]:
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import LSTMStateTuple

class Model():
    def __init__(self, model_params, tSize, global_step=None):
        # super(Model, self).__init__(model_params, global_step)
        self.model_params = model_params
        self._global_step = global_step

        # ---- placeholders ---- #
        self._tSize = tSize
        self._macro_feature_dim = self.model_params['macro_feature_dim']
        self._individual_feature_dim = self.model_params['individual_feature_dim']
        self._I_macro_placeholder = tf.placeholder(dtype=tf.float32, shape=[self._tSize, self._macro_feature_dim], name='macroFeaturePlaceholder')
        self._I_placeholder = tf.placeholder(dtype=tf.float32, shape=[self._tSize, None, self._individual_feature_dim], name='individualFeaturePlaceholder')
        self._R_placeholder = tf.placeholder(dtype=tf.float32, shape=[self._tSize, None], name='returnPlaceholder')
        self._mask_placeholder = tf.placeholder(dtype=tf.bool, shape=[self._tSize, None], name='maskPlaceholder')
        self._dropout_placeholder = tf.placeholder_with_default(1.0, shape=[], name='Dropout')
        self._nSize = tf.shape(self._R_placeholder)[1]
        
        
        # initialize
        # self._state_size = initial_state_size(self.model_params['cell_type_rnn'], self.model_params['num_units_rnn'])
        # self._initial_state_placeholder = tf.placeholder(dtype=tf.float32, shape=[1, self._state_size])
        # splits = [2*unit for unit in self.model_params['num_units_rnn']]
        # self._initial_state = tuple([LSTMStateTuple(*tf.split(value=layer_state, num_or_size_splits=2, axis=1))
        #         for layer_state in tf.split(value=self._initial_state_placeholder, num_or_size_splits=splits, axis=1)])
        # self._initial_state = self._initial_state[0]

        self._rnn_input = tf.expand_dims(self._I_macro_placeholder, axis=0)

        with tf.variable_scope(name_or_scope='Model_Layer'):
            self.build()
        


    def build(self):

        with tf.variable_scope('RNN_Layer'):
            rnn_cell = create_rnn_cell(
                        cell_type=self.model_params['cell_type_rnn'],
                        num_units=self.model_params['num_units_rnn'],
                        num_layers=self.model_params['num_layers_rnn'],
                        dp_input_keep_prob=self._dropout_placeholder,
                        dp_output_keep_prob=1.0
                        )
            print(rnn_cell)

            rnn_outputs, rnn_state = tf.nn.dynamic_rnn(
                        cell=rnn_cell,
                        inputs=self._rnn_input,
                        # initial_state=self._initial_state,
                        dtype=tf.float32)
            
            self._macro_nn_input = tf.squeeze(rnn_outputs, axis=0) # RNN 통과해서 나온 macro 변수들의 hidden state?

        with tf.variable_scope('NN_Layer'):
            I_macro_tile = tf.tile(tf.expand_dims(self._macro_nn_input, axis=1), [1,self._nSize,1]) # T * N * macro_feature_dim
            I_macro_masked = tf.boolean_mask(I_macro_tile, mask=self._mask_placeholder)
            I_masked = tf.boolean_mask(self._I_placeholder, mask=self._mask_placeholder)
            I_concat = tf.concat([I_masked, I_macro_masked], axis=1) # None * (macro_feature_dim + individual_feature_dim)
            R_masked = tf.boolean_mask(self._R_placeholder, mask=self._mask_placeholder)

            h_l = I_concat # 이게 macro와 firm-spec.이 합쳐진 것

            with tf.variable_scope('dense_layer_1'):
                layer_l = Dense(units=64, activation=tf.nn.relu)
                h_l = layer_l(h_l) # macro와 firm-spec.이 합쳐진 것이 Dense layer를 통과함 
                h_l = tf.nn.dropout(h_l, self._dropout_placeholder)

            
            with tf.variable_scope('last_dense_layer'):
                layer = Dense(units=1) # 마지막 dense layer이므로 output unit 개수가 1
                R_pred = layer(h_l)
                self._R_pred = tf.reshape(R_pred, shape=[-1])

        self._loss = tf.reduce_mean(tf.square(R_masked - self._R_pred))
        self._train_model_op = tf.contrib.layers.optimize_loss(
                                                                loss=self._loss,
                                                                global_step=self._global_step,
                                                                learning_rate=0.01,
                                                                optimizer="Adam")


    def train(self, sess, dl, dl_valid):
        sharpe_train = [] ; loss_train = []
        sharpe_valid = []
        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        sess.run(init_g)
        sess.run(init_l)
        
        for epoch in range(20):
            print('Epoch %d' %epoch)
            for _, (I_macro, I, R, mask) in enumerate(dl.iterateOneEpoch(subEpoch=False)):

                fetches = [self._train_model_op]
                feed_dict = {self._I_macro_placeholder:I_macro,
                            self._I_placeholder:I,
                            self._R_placeholder:R,
                            self._mask_placeholder:mask,
                            self._dropout_placeholder:self.model_params['dropout']
                            }
                
                sess.run(fetches=fetches, feed_dict=feed_dict)
            
            ### evaluate train loss / sharpe
            train_epoch_loss = self.evaluate_loss(sess, dl)
            train_epoch_r2 = self.evaluate_r_sq(sess, dl)
            print('train loss: ', train_epoch_loss)
            print('train R-Squared: ', train_epoch_r2)
            loss_train.append(train_epoch_loss)

    def evaluate_loss(self, sess, dl):
        for _, (I_macro, I, R, mask) in enumerate(dl.iterateOneEpoch(subEpoch=False)):
            feed_dict = {self._I_macro_placeholder:I_macro,
                        self._I_placeholder:I,
                        self._R_placeholder:R,
                        self._mask_placeholder:mask,
                        self._dropout_placeholder:1.0
                            }
            loss, = sess.run([self._loss], feed_dict=feed_dict)
        return loss

    def getPrediction(self, sess, dl):
        for _, (I_macro, I, R, mask) in enumerate(dl.iterateOneEpoch(subEpoch=False)):
            feed_dict = {self._I_macro_placeholder:I_macro,
                        self._I_placeholder:I,
                        self._R_placeholder:R,
                        self._mask_placeholder:mask,
                        self._dropout_placeholder:1.0}
            R_pred, = sess.run(fetches=[self._R_pred], feed_dict=feed_dict)
        return R_pred

    def evaluate_r_sq(self, sess, dl):
        R_pred = self.getPrediction(sess, dl) ; print(R_pred.shape)
        for _, (I_macro, I, R, mask) in enumerate(dl.iterateOneEpoch(subEpoch=False)):
            r_sq = 1 - ( sum((R[mask] - R_pred)**2) / sum(R[mask]**2) )
        return r_sq

In [140]:
tf.reset_default_graph()
global_step = tf.train.get_or_create_global_step()
model = Model(model_params=config, tSize=240, global_step=global_step)

In [141]:
sess = tf.Session()
model.train(sess, dl, dl_valid)

Epoch 0
(336113,)
train loss:  0.030410819
train R-Squared:  -0.8283528237461184
Epoch 1
(336113,)
train loss:  0.02918724
train R-Squared:  -0.754789152138359
Epoch 2
(336113,)
train loss:  0.02674354
train R-Squared:  -0.6078695944862302
Epoch 3
(336113,)
train loss:  0.024638131
train R-Squared:  -0.481288522257006
Epoch 4
(336113,)
train loss:  0.021807054
train R-Squared:  -0.31107920566575387
Epoch 5
(336113,)
train loss:  0.0201662
train R-Squared:  -0.21242807322760138
Epoch 6
(336113,)
train loss:  0.01953604
train R-Squared:  -0.1745417696542093
Epoch 7
(336113,)
train loss:  0.019178407
train R-Squared:  -0.1530402077057138
Epoch 8
(336113,)
train loss:  0.018774021
train R-Squared:  -0.1287277658872461
Epoch 9
(336113,)
train loss:  0.01820862
train R-Squared:  -0.0947347489843513
Epoch 10
(336113,)
train loss:  0.017426161
train R-Squared:  -0.04769199653145151
Epoch 11
(336113,)
train loss:  0.016581897
train R-Squared:  0.0030667276273956823
Epoch 12
(336113,)
train loss